In [17]:
import pandas as pd
import numpy as np
import re
import nltk
import string
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split


import seaborn as sns
%matplotlib inline

pal = sns.color_palette()

/Users/Diane/Desktop/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [55]:
train = pd.read_csv('train.csv').dropna()

In [4]:
train.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [5]:
test = pd.read_csv('test.csv').dropna()

In [6]:
test.head()

,test_id,question1,question2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?
2,2,What but is the best way to send money from Ch...,What you send money to China?
3,3,Which food not emulsifiers?,What foods fibre?
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?


In [7]:
# train_data, validation_data = train_test_split(train, test_size = 0.2)

In [8]:
stemmer = SnowballStemmer("english", ignore_stopwords=True)

In [9]:
def build_vocab(datasets):
  vocab = dict()
  
  for text in datasets: 
    text = text.lower().split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    
    text = re.sub("[^\w]", " ",  text)
#     text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
   
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    
    for word in text:
        if word not in vocab:
            vocab[word] = len(vocab)

  print('Vocab size: {}'.format(len(vocab)))

  return vocab

In [10]:
def build_vocab1(datasets):
  vocab = dict()
  
  for data in datasets:
        wordList = re.sub("[^\w]", " ",  data).lower().split()
#         wordList = [word for word in wordList if word not in stopwords.words('english')]
        for word in wordList:
            word = stemmer.stem(word)
            if word not in vocab and word not in stopwords.words('english'):
                vocab[word] = len(vocab)

  print('Vocab size: {}'.format(len(vocab)))

  return vocab

In [199]:
total_vocab = build_vocab(np.append(train_data.question1, train_data.question2))

Vocab size: 77597


In [201]:
total_vocab1 = build_vocab1(np.append(train_data.question1, train_data.question2))

Vocab size: 58473


In [7]:
# rebalance data

In [58]:
def word_match_share(row):
    q1words = {}
    q2words = {}
    for word in re.sub("[^\w]", " ",  row['question1']).lower().split():
#     for word in str(row['question1']).lower().split():
        word = stemmer.stem(word)
        if word not in stopwords.words('english'):
            q1words[word] = 1
    for word in re.sub("[^\w]", " ",  row['question2']).lower().split():
#     for word in str(row['question2']).lower().split():
        word = stemmer.stem(word)
        if word not in stopwords.words('english'):
            q2words[word] = 1
    if len(q1words) == 0 or len(q2words) == 0:
        return 0
    shared_words_in_q1 = [w for w in q1words.keys() if w in q2words]
    shared_words_in_q2 = [w for w in q2words.keys() if w in q1words]
    R = float(len(shared_words_in_q1) + len(shared_words_in_q2))/(len(q1words) + len(q2words))
    return R

In [25]:
train_word_match = train.apply(word_match_share, axis=1, raw=True)

/Users/Diane/Desktop/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
/Users/Diane/Desktop/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:10: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


In [59]:
train_word_match1 = train.apply(word_match_share, axis=1, raw=True)

In [69]:
x_train = pd.DataFrame()
x_test = pd.DataFrame()
x_train['word_match'] = train_word_match
# x_test['word_match'] = test.apply(word_match_share, axis=1, raw=True)
y_train = train['is_duplicate'].values

In [70]:
pos_train = x_train[y_train == 1]
neg_train = x_train[y_train == 0]

In [71]:
# reference from https://www.kaggle.com/anokas/data-analysis-xgboost-starter-0-35460-lb

p = 0.165
scale = ((len(pos_train) / float(len(pos_train) + len(neg_train))) / p) - 1
while scale > 1:
    neg_train = pd.concat([neg_train, neg_train])
    scale -=1
neg_train = pd.concat([neg_train, neg_train[:int(scale * len(neg_train))]])
print(len(pos_train) / float(len(pos_train) + len(neg_train)))

x_train = pd.concat([pos_train, neg_train])
y_train = (np.zeros(len(pos_train)) + 1).tolist() + np.zeros(len(neg_train)).tolist()
del pos_train, neg_train

0.19124341597


In [72]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2)

In [78]:
import xgboost as xgb

params = {}
params['objective'] = 'binary:logistic'
params['eval_metric'] = 'logloss'
params['eta'] = 0.02
params['max_depth'] = 6

d_train = xgb.DMatrix(x_train, label=y_train)
d_valid = xgb.DMatrix(x_valid, label=y_valid)

watchlist = [(d_train, 'train'), (d_valid, 'valid')]

bst = xgb.train(params, d_train, 400, watchlist, early_stopping_rounds=50, verbose_eval=10)

[0]	train-logloss:0.683169	valid-logloss:0.683231
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
[10]	train-logloss:0.60184	valid-logloss:0.602426
[20]	train-logloss:0.54452	valid-logloss:0.545512
[30]	train-logloss:0.502731	valid-logloss:0.504048
[40]	train-logloss:0.471566	valid-logloss:0.473149
[50]	train-logloss:0.447946	valid-logloss:0.44975
[60]	train-logloss:0.429706	valid-logloss:0.431711
[70]	train-logloss:0.415598	valid-logloss:0.417775
[80]	train-logloss:0.404598	valid-logloss:0.406922
[90]	train-logloss:0.395972	valid-logloss:0.398417
[100]	train-logloss:0.389181	valid-logloss:0.391734
[110]	train-logloss:0.383827	valid-logloss:0.386483
[120]	train-logloss:0.379595	valid-logloss:0.382339
[130]	train-logloss:0.376243	valid-logloss:0.379063
[140]	train-logloss:0.373561	valid-logloss:0.376451
[150]	train-logloss:0.371418	valid-logloss:0.374374
[160]	train-logloss:0.369707	va

In [74]:
d_val = xgb.DMatrix(x_valid)
p_val = bst.predict(d_val)

# val = pd.DataFrame(index = x_valid.index)
# val['is_duplicate'] = p_val

In [75]:
for i in range(len(p_val)):
    if p_val[i] >= 0.5:
        p_val[i] = 1
    else: 
        p_val[i] = 0

In [76]:
count = 0

for i in range(len(p_val)):
    if p_val[i] == y_valid[i]:
        count += 1

In [77]:
float(count)/len(p_val)

0.8122013094338172

In [93]:
def clean_sentence(sent):
    sentence = re.sub("[^\w]", " ",  sent).lower().split()

    for word in sentence:
#         word = stemmer.stem(word)
        if word in stopwords.words('english'):
            sentence.remove(word)

    sentence = " ".join(sentence)
    return sentence

In [94]:
def cosine(v1, v2):
    
    v1 = np.array(v1)
    v2 = np.array(v2)

    return np.dot(v1, v2) / (np.sqrt(np.sum(v1**2)) * np.sqrt(np.sum(v2**2)))

In [95]:
for col in ['question1', 'question2']:
    train_data[col] = train_data[col].apply(clean_sentence)

/Users/Diane/Desktop/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [113]:
print len(train_data)
train_data.head()

323429


,id,qid1,qid2,question1,question2,is_duplicate
286569,286569,324559,190333,does linkedin let see viewed profile,facebook ever offer feature like linkedin wher...,0
291655,291655,19999,68878,people 99 be allowed vote,people 97 be allowed vote,1
139515,139515,102850,221939,is best way write cv,can write cv,1
56117,56117,98814,98815,is shah rukh khan real life,shah rukh khan real life what is it like inter...,1
73504,73504,126130,126131,makes leader,makes leader leader,1


In [114]:
train_questions = train_data['question1'].append(train_data['question2'], ignore_index=True)    

In [115]:
train_questions.head()

0    does linkedin let see viewed profile
1               people 99 be allowed vote
2                    is best way write cv
3             is shah rukh khan real life
4                            makes leader
dtype: object